<a id="ref3"></a>

# Content-Based recommendation system

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

%matplotlib inline

%load_ext autoreload
%autoreload 2

import MLearningUtils as mlUtil


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
df_BookRatings = pd.read_csv('data/BX-Book-Ratings.csv', delimiter=";")
df_BookRatings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [12]:
util = mlUtil.MLearningUtils()
util.check_missing_data(df_BookRatings)

Qtd rows X columns:(1149780, 3)

Are there any Missing values?:False

Total of missing ROWS values: 0

Has duplicated values? how to drop => df=df.drop_duplicates() 
False    1149780
dtype: int64

Missing values by columns:
User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

Total sum of missing values: 0

Percentage of missing values for each variable:
User-ID        0.0
ISBN           0.0
Book-Rating    0.0
dtype: float64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB
None


In [ ]:
# df_Books = pd.read_csv('data/BX-Books.csv', delimiter=";")
# print(df_Books.head(20))

In [13]:
df_Books = pd.read_csv('data/BX-Books2.csv')
df_Books.head()

,"ISBN;""Book-Title"";""Book-Author"";""Year-Of-Publication"";""Publisher"";""Image-URL-S"";""Image-URL-M"";""Image-URL-L"""
0,"0195153448;""Classical Mythology"";""Mark P. O. M..."
1,"0002005018;""Clara Callan"";""Richard Bruce Wrigh..."
2,"0060973129;""Decision in Normandy"";""Carlo D'Est..."
3,"0374157065;""Flu: The Story of the Great Influe..."
4,"0393045218;""The Mummies of Urumchi"";""E. J. W. ..."


In [14]:
for i in range(10,11):    
    value = df_Books['ISBN;"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"'][i].split(';')
value

['0771074670',
 '"Nights Below Station Street"',
 '"David Adams Richards"',
 '"1988"',
 '"Emblem Editions"',
 '"http://images.amazon.com/images/P/0771074670.01.THUMBZZZ.jpg"',
 '"http://images.amazon.com/images/P/0771074670.01.MZZZZZZZ.jpg"',
 '"http://images.amazon.com/images/P/0771074670.01.LZZZZZZZ.jpg"']

In [15]:
df_Books.columns

Index(['ISBN;"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"'], dtype='object')

In [16]:
# Rename the "big" column name to: 'cols_to_split' 
df_Books.rename(columns={'ISBN;"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"': 'cols_to_split'}, inplace=True)


In [17]:
print(len(df_Books.iloc[9068]['cols_to_split'].split(";")))
df_Books.iloc[9068]['cols_to_split'].split(";")

10


['1857033701',
 '"Getting into Films &amp',
 ' Television: How to Find the Best Way in (Jobs &amp',
 ' Careers)"',
 '"Robert Angell"',
 '"1997"',
 '"Trans-Atlantic Publications"',
 '"http://images.amazon.com/images/P/1857033701.01.THUMBZZZ.jpg"',
 '"http://images.amazon.com/images/P/1857033701.01.MZZZZZZZ.jpg"',
 '"http://images.amazon.com/images/P/1857033701.01.LZZZZZZZ.jpg",,,,,,,,,,,,,,,,']

In [18]:
print(len(df_Books.iloc[11]['cols_to_split'].split(";")))
df_Books.iloc[11]['cols_to_split'].split(";")

8


['080652121X',
 '"Hitler\'s Secret Bankers: The Myth of Swiss Neutrality During the Holocaust"',
 '"Adam Lebor"',
 '"2000"',
 '"Citadel Press"',
 '"http://images.amazon.com/images/P/080652121X.01.THUMBZZZ.jpg"',
 '"http://images.amazon.com/images/P/080652121X.01.MZZZZZZZ.jpg"',
 '"http://images.amazon.com/images/P/080652121X.01.LZZZZZZZ.jpg"']

In [19]:
df_Books['ISBN'] = df_Books['cols_to_split'].str.split(';').str[0]
df_Books['Title'] = df_Books['cols_to_split'].str.split(';').str[1].str.strip('"')
df_Books['Author'] = df_Books['cols_to_split'].str.split(';').str[2].str.strip('"')
df_Books['Year'] = df_Books['cols_to_split'].str.split(';').str[3].str.strip('"')

In [20]:
df_Books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   cols_to_split  271379 non-null  object
 1   ISBN           271379 non-null  object
 2   Title          271379 non-null  object
 3   Author         234439 non-null  object
 4   Year           232533 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB


In [21]:
len(df_Books['ISBN'].unique())

271378

In [22]:
df_Books.columns

Index(['cols_to_split', 'ISBN', 'Title', 'Author', 'Year'], dtype='object')

In [23]:
# Re-order columns
df_Books = df_Books[['ISBN', 'Title', 'Author', 'Year','cols_to_split']]

In [24]:
df_Books.head()

,ISBN,Title,Author,Year,cols_to_split
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,"0195153448;""Classical Mythology"";""Mark P. O. M..."
1,0002005018,Clara Callan,Richard Bruce Wright,2001,"0002005018;""Clara Callan"";""Richard Bruce Wrigh..."
2,0060973129,Decision in Normandy,Carlo D'Este,1991,"0060973129;""Decision in Normandy"";""Carlo D'Est..."
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,"0374157065;""Flu: The Story of the Great Influe..."
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,"0393045218;""The Mummies of Urumchi"";""E. J. W. ..."


In [25]:
#Merge dataframes to create input for recommender systems
df_Ratings = pd.merge(df_BookRatings, df_Books.drop_duplicates(['ISBN']), on="ISBN", how="left")

In [26]:
df_Ratings.head()

,User-ID,ISBN,Book-Rating,Title,Author,Year,cols_to_split
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,"034545104X;""Flesh Tones: A Novel"";""M. J. Rose""..."
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,"0155061224;""Rites of Passage"";""Judith Rae"";""20..."
2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,"0446520802;""The Notebook"";""Nicholas Sparks"";""1..."
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,"052165615X;""Help!: Level 1"";""Philip Prowse"";""1..."
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,"0521795028;""The Amsterdam Connection : Level 4..."


In [27]:
df_Ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1149780 entries, 0 to 1149779
Data columns (total 7 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   User-ID        1149780 non-null  int64 
 1   ISBN           1149780 non-null  object
 2   Book-Rating    1149780 non-null  int64 
 3   Title          1031175 non-null  object
 4   Author         917998 non-null   object
 5   Year           914116 non-null   object
 6   cols_to_split  1031175 non-null  object
dtypes: int64(2), object(5)
memory usage: 70.2+ MB


In [ ]:
# the most popular Books in the dataset

# book_grouped = df_Ratings.groupby(['ISBN']).agg({'Book-Rating': 'count'}).reset_index()
# grouped_sum = book_grouped['Book-Rating'].sum()
# book_grouped['percentage']  = book_grouped['listen_count'].div(grouped_sum)*100
# book_grouped.sort_values(['Book-Rating', 'ISBN'], ascending = [0,1])


In [ ]:
# max/min rating values
print(df_Ratings['Book-Rating'].max())
print(df_Ratings['Book-Rating'].min())

In [ ]:
# Count number of unique users in the dataset
users = df_Ratings['User-ID'].unique()
len(users)


In [ ]:
# number of unique books in the dataset
books = df_Ratings['ISBN'].unique()
len(books)

In [118]:
# rank by number of ratings
book_grouped = df_Ratings.groupby(['ISBN', 'Title', 'Author', 'Year']).agg({'Book-Rating': 'count'}).reset_index()
book_grouped.sort_values(['Book-Rating', 'ISBN'], ascending = [0,1]).head(10)

,ISBN,Title,Author,Year,Book-Rating
183449,0971880107,Wild Animus,Rich Shapero,2004,2502
34309,0316666343,The Lovely Bones: A Novel,Alice Sebold,2002,1295
56348,0385504209,The Da Vinci Code,Dan Brown,2003,883
6487,0060928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,732
28718,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,723
70944,044023722X,A Painted House,John Grisham,2001,647
18977,0142001740,The Secret Life of Bees,Sue Monk Kidd,2003,615
120906,067976402X,Snow Falling on Cedars,David Guterson,1995,614
110403,0671027360,Angels &amp,Demons,Dan Brown,586
76812,0446672211,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,1998,585


In [ ]:
# the wrost book based on ratings
df_Ratings[df_Ratings['ISBN']=='0971880107']['Book-Rating'].value_counts()

In [ ]:
# rank by sum of ratings
book_grouped = df_Ratings.groupby(['ISBN', 'Title', 'Author', 'Year']).agg({'Book-Rating': 'sum'}).reset_index()
book_grouped.sort_values(['Book-Rating', 'ISBN'], ascending = [0,1]).head(10)

In [ ]:
df_Ratings[df_Ratings['ISBN']=='0385504209']['Book-Rating'].value_counts()

In [ ]:
df_Ratings.columns

In [30]:
# get only books who has ratings between 5 and 10
print(df_Ratings[df_Ratings['Book-Rating'] >=5 ].shape)

df_Ratings_selected = df_Ratings[df_Ratings['Book-Rating']>=5]

#df_Ratings_selected.reset_index(drop=True)


(414242, 7)


In [191]:
df_Ratings_selected.head()

,index,User-ID,ISBN,Book-Rating,Title,Author,Year,cols_to_split
0,1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,"0155061224;""Rites of Passage"";""Judith Rae"";""20..."
1,4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,"0521795028;""The Amsterdam Connection : Level 4..."
2,6,276736,3257224281,8,NaN,NaN,NaN,NaN
3,7,276737,0600570967,6,NaN,NaN,NaN,NaN
4,8,276744,038550120X,7,A Painted House,JOHN GRISHAM,2001,"038550120X;""A Painted House"";""JOHN GRISHAM"";""2..."


In [192]:
# rank by sum of ratings
book_grouped = df_Ratings_selected.groupby(['ISBN', 'Title', 'Author', 'Year']).agg({'Book-Rating': 'sum'}).reset_index()
book_grouped.sort_values(['Book-Rating', 'ISBN'], ascending = [0,1]).head(20)

,ISBN,Title,Author,Year,Book-Rating
19029,0316666343,The Lovely Bones: A Novel,Alice Sebold,2002,5743
31243,0385504209,The Da Vinci Code,Dan Brown,2003,4074
15627,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,3086
57162,059035342X,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,2794
10737,0142001740,The Secret Life of Bees,Sue Monk Kidd,2003,2570
3545,0060928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,2496
42565,0446672211,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,1998,2382
47692,0452282152,Girl with a Pearl Earring,Tracy Chevalier,2001,2199
60810,0671027360,Angels &amp,Demons,Dan Brown,2161
18987,0316601950,The Pilot's Wife : A Novel,Anita Shreve,1999,1999


In [193]:
book_grouped['Author'].value_counts().head(10)

Agatha Christie        390
Stephen King           379
William Shakespeare    339
Terry Pratchett        182
Nora Roberts           176
Charles Dickens        175
Isaac Asimov           159
Barbara Cartland       154
Piers Anthony          141
Jane Austen            138
Name: Author, dtype: int64

In [194]:
df_Ratings_selected[(df_Ratings_selected['ISBN']=='0316666343') & (df_Ratings_selected['Book-Rating']==10)].head()

,index,User-ID,ISBN,Book-Rating,Title,Author,Year,cols_to_split
2870,11078,638,0316666343,10,The Lovely Bones: A Novel,Alice Sebold,2002,"0316666343;""The Lovely Bones: A Novel"";""Alice ..."
4312,13900,1928,0316666343,10,The Lovely Bones: A Novel,Alice Sebold,2002,"0316666343;""The Lovely Bones: A Novel"";""Alice ..."
7488,20801,4017,0316666343,10,The Lovely Bones: A Novel,Alice Sebold,2002,"0316666343;""The Lovely Bones: A Novel"";""Alice ..."
13465,34882,8253,0316666343,10,The Lovely Bones: A Novel,Alice Sebold,2002,"0316666343;""The Lovely Bones: A Novel"";""Alice ..."
14204,36834,8801,0316666343,10,The Lovely Bones: A Novel,Alice Sebold,2002,"0316666343;""The Lovely Bones: A Novel"";""Alice ..."


In [195]:
# 8253

#Get a count of user_ids for each unique book as recommendation score
data_grouped = df_Ratings_selected.groupby(['ISBN', 'Title', 'Author', 'Year']).agg({'User-ID': 'count'}).reset_index()
data_grouped.rename(columns = {'User-ID': 'Qtd_Users'},inplace=True)

#Sort the songs based upon recommendation score
data_sort = data_grouped.sort_values(['Qtd_Users', 'ISBN'], ascending = [0,1])

# data_grouped.head()

data_sort.head()


,ISBN,Title,Author,Year,Qtd_Users
19029,0316666343,The Lovely Bones: A Novel,Alice Sebold,2002,692
31243,0385504209,The Da Vinci Code,Dan Brown,2003,475
15627,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,369
3545,0060928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,312
57162,059035342X,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,310


In [ ]:
#rank based upon score
# data_sort['Rank'] = data_sort['score'].rank(ascending=0, method='first')
# data_sort.head(10)


In [113]:
#1. Get all unique items for a user

#############################################################
# Get unique books from user by Book-Rating
#############################################################

user_data1 = df_Ratings_selected[df_Ratings_selected['User-ID'] == 8253]
user_data1.sort_values(['Book-Rating', 'ISBN'], ascending = [0,1]).head()


,User-ID,ISBN,Book-Rating,Title,Author,Year,cols_to_split
34841,8253,0060928336,10,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,"0060928336;""Divine Secrets of the Ya-Ya Sister..."
34843,8253,0060929871,10,Brave New World,Aldous Huxley,1998,"0060929871;""Brave New World"";""Aldous Huxley"";""..."
34844,8253,0060930535,10,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,"0060930535;""The Poisonwood Bible: A Novel"";""Ba..."
34851,8253,0064473511,10,A Tale of Time City,Diana Wynne Jones,2002,"0064473511;""A Tale of Time City"";""Diana Wynne ..."
34853,8253,0140157379,10,Haroun and the Sea of Stories,Salman Rushdie,1991,"0140157379;""Haroun and the Sea of Stories"";""Sa..."


In [34]:
%load_ext autoreload
%autoreload 2

# Custom class for similarity recommendation model 
import recommender as recommender

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
columns_to_append_on_results = ['ISBN','Title', 'Author']
max_unique_data_items_to_compare = 1000

rec1 = recommender.recommender_by_similarity()

result1 = rec1.recommend(
                8253,
               'User-ID','ISBN',
               df_Ratings_selected, 
               columns_to_append_on_results, 
               max_unique_data_items_to_compare)

result1

No. of unique items for the user: 76
No. of unique items to be used for the recommendation dataset: 1000


Processing: 100%|██████████| 1000/1000 [00:19<00:00, 50.74it/s]


(76, 1000)
Non zero values in cooccurence_matrix :29101


,user_id,ISBN,score,rank,Title,Author
0,8253,0439064864,0.012265,1,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling
1,8253,0452269571,0.009659,2,Bastard Out of Carolina,Dorothy Allison
2,8253,014029628X,0.009561,3,Girl in Hyacinth Blue,Susan Vreeland
3,8253,0439136350,0.009092,4,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling
4,8253,0385722435,0.009057,5,Ella Minnow Pea: A Novel in Letters,Mark Dunn
5,8253,0385722206,0.008576,6,Balzac and the Little Chinese Seamstress : A N...,DAI SIJIE
6,8253,0439139597,0.008570,7,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling
7,8253,006000438X,0.008484,8,The Death of Vishnu: A Novel,Manil Suri
8,8253,0446391301,0.008377,9,Geek Love,Katherine Dunn
9,8253,0679751521,0.008318,10,Midnight in the Garden of Good and Evil,John Berendt


In [33]:
rec2 = recommender.recommender_by_similarity()

rec2.recommend(8253,
               'User-ID','ISBN',
               df_Ratings_selected, 
               columns_to_append_on_results, 
               5000)

Processing: 100%|██████████| 5000/5000 [01:38<00:00, 50.56it/s]


(76, 5000)
No. of unique items for the user: 76
No. of unique items to be used for the recommendation dataset: 5000
Non zero values in cooccurence_matrix :83829


,user_id,ISBN,score,rank,Title,Author
0,8253,0439064864,0.012265,1,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling
1,8253,0061093106,0.010070,2,"Red,"" White and Blue : A Novel"""",""Susan Isaacs...",NaN
2,8253,0385494149,0.009858,3,Enduring Love,Ian McEwan
3,8253,0440224810,0.009743,4,Special Delivery: A Novel,Danielle Steel
4,8253,0452269571,0.009659,5,Bastard Out of Carolina,Dorothy Allison
5,8253,014029628X,0.009561,6,Girl in Hyacinth Blue,Susan Vreeland
6,8253,0060540753,0.009547,7,Patron Saint of Liars : A Novel,Ann Patchett
7,8253,0440931800,0.009508,8,From the Mixed Up Files of Mrs. Basil E. Frank...,E. L. Konigsburg
8,8253,0156001314,0.009507,9,The Name of the Rose: including Postscript to ...,Umberto Eco
9,8253,0394756886,0.009474,10,"Bright lights,"" big city: A novel"""",""Jay McIne...",NaN


In [100]:
rec2.recommend(8253,
               'User-ID','ISBN',
               df_Ratings_selected, 
               columns_to_append_on_results, 
               10000)

Progress: [####################] 100.0%
(76, 10000)
No. of unique items for the user: 76
No. of unique items to be used for the recommendation dataset: 10000
Non zero values in cooccurence_matrix :121243


,user_id,ISBN,score,rank,Title,Author
0,8253,0439064864,0.012265,1,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling
1,8253,0316313068,0.011716,2,In the Land of Dreamy Dreams,Ellen Gilchrist
2,8253,0380712040,0.011480,3,Black Cherry Blues: A Dave Robicheaux Novel,James Lee Burke
3,8253,0805055908,0.011419,4,The Enchantment of Lily Dahl: A Novel,Siri Hustvedt
4,8253,0451405498,0.011247,5,Anna's Book,Ruth Rendell
5,8253,0061093092,0.010750,6,Lily White,Susan Isaacs
6,8253,0385333293,0.010723,7,A Bigamist's Daughter,ALICE MCDERMOTT
7,8253,0553562754,0.010652,8,Southern Ghost,Carolyn G. Hart
8,8253,0811831620,0.010554,9,The Gryphon: In Which the Extraordinary Corres...,Sabine Is Rediscovered
9,8253,0451200152,0.010553,10,The Town,Bentley Little


In [202]:
# David McAlister Barry is an American author and columnist who wrote a nationally syndicated humor column for the Miami Herald from 1983 to 2005.
author_DavedBarry_Ratings = df_Ratings_selected[(df_Ratings_selected['Author']=='Dave Barry') ]
print(author_DavedBarry_Ratings.shape)
author_DavedBarry_Ratings.sort_values(['ISBN', 'Book-Rating'], ascending = [0,1]).head(20)


(331, 7)


,User-ID,ISBN,Book-Rating,Title,Author,Year,cols_to_split
935344,226851,3821830824,7,Big Trouble.,Dave Barry,2001,"3821830824;""Big Trouble."";""Dave Barry"";""2001"";..."
208136,48052,3821809337,6,Die Achse des Bl�?¶den. Eine politische Evolut...,Dave Barry,2003,"3821809337;""Die Achse des Bl�?¶den. Eine polit..."
826681,199827,3404921135,7,Jede Menge �?�?rger. Big Trouble.,Dave Barry,2002,"3404921135;""Jede Menge �?�?rger. Big Trouble.""..."
1136263,273203,3404921135,8,Jede Menge �?�?rger. Big Trouble.,Dave Barry,2002,"3404921135;""Jede Menge �?�?rger. Big Trouble.""..."
284960,68066,1587880245,6,Big Trouble,Dave Barry,2001,"1587880245;""Big Trouble"";""Dave Barry"";""2001"";""..."
444782,106595,1579544355,6,All the Dave Barry You Could Ever Want,Dave Barry,2001,"1579544355;""All the Dave Barry You Could Ever ..."
330932,78783,0878577254,8,Dave Barry's Guide to Marriage and/or Sex,Dave Barry,2000,"0878577254;""Dave Barry's Guide to Marriage and..."
496973,119654,0878576525,7,Claw Your Way to the Top: How to Become the He...,Dave Barry,2000,"0878576525;""Claw Your Way to the Top: How to B..."
175098,37950,0878576525,8,Claw Your Way to the Top: How to Become the He...,Dave Barry,2000,"0878576525;""Claw Your Way to the Top: How to B..."
64704,13173,0878575707,5,Dave Barry's Stay Fit and Healthy Until You're...,Dave Barry,2000,"0878575707;""Dave Barry's Stay Fit and Healthy ..."


In [140]:
# User = 215588	0836214439	9	Dave Barry's Book of Bad Songs	Dave Barry	1997

columns_to_append_on_results = ['ISBN','Title', 'Author']
max_unique_data_items_to_compare = 3000

recommend_user215588 = recommender.recommender_by_similarity()

user215588 = recommend_user215588.recommend(
                215588,
               'User-ID','ISBN',
               df_Ratings_selected, 
               columns_to_append_on_results, 
               max_unique_data_items_to_compare)

user215588

Progress: [####################] 100.0%
(2, 3000)
No. of unique items for the user: 2
No. of unique items to be used for the recommendation dataset: 3000
Non zero values in cooccurence_matrix :576


,user_id,ISBN,score,rank,Title,Author
0,215588,0380752891,0.055714,1,"Man from Mundania (Xanth Trilogy,"" No 12)"""",""P...",NaN
1,215588,0451161343,0.041071,2,Thinner,Stephen King
2,215588,0505525526,0.035897,3,Single White Vampire,Lynsay Sands
3,215588,0451190521,0.035756,4,The Green Mile: The Mouse on the Mile (Green M...,Stephen King
4,215588,0451170113,0.035714,5,Night Shift,Stephen King
5,215588,0743222008,0.034483,6,The Vanished Man : A Lincoln Rhyme Novel,Jeffery Deaver
6,215588,0425157539,0.033482,7,Sudden Prey,John Sandford
7,215588,034538430X,0.033482,8,TickTock,Dean R. Koontz
8,215588,0836218833,0.033333,9,Attack Of The Deranged Mutant Killer Snow Goons,Bill Watterson
9,215588,034530988X,0.031250,10,Friday,Robert Heinlein


In [148]:
user_data2 = df_Ratings_selected[df_Ratings_selected['User-ID'] == 37950]
user_data2.sort_values(['Book-Rating', 'ISBN'], ascending = [0,1]).head(10)

,User-ID,ISBN,Book-Rating,Title,Author,Year,cols_to_split
174501,37950,031210491X,10,Living With Dickens,Tom Bianchi,1994,"031210491X;""Living With Dickens"";""Tom Bianchi""..."
174524,37950,0316107298,10,Billy and the Boingers Bootleg (Bloom County B...,Berke Breathed,1987,"0316107298;""Billy and the Boingers Bootleg (Bl..."
174525,37950,0316107352,10,Tales Too Ticklish to Tell: Bloom County,Berke Breathed,1988,"0316107352;""Tales Too Ticklish to Tell: Bloom ..."
174547,37950,0345356578,10,A Man Rides Through (Man Rides Through),Stephen Donaldson,1990,"0345356578;""A Man Rides Through (Man Rides Thr..."
174718,37950,0440215625,10,Dragonfly in Amber,DIANA GABALDON,1993,"0440215625;""Dragonfly in Amber"";""DIANA GABALDO..."
174786,37950,0449908100,10,Dave Barry Does Japan,Dave Barry,1993,"0449908100;""Dave Barry Does Japan"";""Dave Barry..."
174791,37950,0451142934,10,Skeleton Crew,Stephen King,1986,"0451142934;""Skeleton Crew"";""Stephen King"";""198..."
174795,37950,0451150325,10,Shining,Stephen King,1981,"0451150325;""Shining"";""Stephen King"";""1981"";""Si..."
174804,37950,0451163524,10,"The Drawing of the Three (The Dark Tower,"" Boo...",NaN,NaN,"0451163524;""The Drawing of the Three (The Dark..."
174805,37950,0451166582,10,The Eyes of the Dragon,Stephen King,2001,"0451166582;""The Eyes of the Dragon"";""Stephen K..."


In [146]:
columns_to_append_on_results = ['ISBN','Title', 'Author']
max_unique_data_items_to_compare = 10000

recommend_user37950 = recommender.recommender_by_similarity()

user37950 = recommend_user37950.recommend(
                37950,
               'User-ID','ISBN',
               df_Ratings_selected, 
               columns_to_append_on_results, 
               max_unique_data_items_to_compare)

user37950

Progress: [####################] 100.0%
(229, 10000)
No. of unique items for the user: 229
No. of unique items to be used for the recommendation dataset: 10000
Non zero values in cooccurence_matrix :256038


,user_id,ISBN,score,rank,Title,Author
0,37950,0345326784,0.011945,1,The Deryni Archives,Katherine Kurtz
1,37950,0345421620,0.011717,2,The Demon Awakens (DemonWars),R. A. Salvatore
2,37950,0380708213,0.011707,3,The Forever War,Joe Haldeman
3,37950,0380769530,0.011634,4,Untamed,Elizabeth Lowell
4,37950,0812585151,0.011385,5,Bard: The Odyssey of the Irish (Celtic World o...,Morgan Llywelyn
5,37950,0451201159,0.011183,6,Murder at the Vicarage (Miss Marple Mysteries ...,Agatha Christie
6,37950,0451132378,0.011001,7,Pet Sematary,Stephen King
7,37950,0451200152,0.010914,8,The Town,Bentley Little
8,37950,0446363073,0.010864,9,Midnight Graffiti,James Van Hise
9,37950,0515103454,0.010771,10,Haunted,James Herbert
